# Homework #3 

Matriculation number: A0153347A
Email: e0025553@nus.edu.sg

### General Notes about this assignment 

Data:
X_train.npy: training data numpy array. Each row in the array corresponds to an image unrolled to a vector (50 x 37 = 1850 dimension)

y_train.npy: labels (0-6) of each data corresponding to the image in the same row in X_train.npy

X_test.npy: testing data numpy array for evaluation of your models. The prediction outputs on this test data is to be submitted for scoreboard

Ideas:
- multiclass SVM (see Multiclass classification)
- kNN with PCA 
- CNN  
- Viola-Jones face detector


### Files included with this submission

In my submission there are this iPython notebook file for the code part and my pdf regarding the essay.

## Building and evaluating models

In [9]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd 
%matplotlib inline

from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from time import time

# Size the plot appropriately for online display
pl.rcParams['figure.figsize'] = (12.0, 10.0)

In [10]:
#Load dataset

#train data

xin= np.load("./X_train.npy")
yin= np.load("./y_train.npy")
xout= np.load("./X_test.npy")

In [11]:
def svmModel(ker, xin,yin, c, g):
    modelOVO= svm.SVC(decision_function_shape='ovo',kernel=ker, C=c, gamma=g)
    modelOVO.fit(xin,yin)
    modelOVR= svm.SVC(decision_function_shape='ovr',kernel=ker, C=c, gamma=g)
    modelOVR.fit(xin,yin)
    return (modelOVO,modelOVR)

In [12]:
def svmPredict(model,x,y):
    ypredict= model.predict(x)
    score= f1_score(y , ypredict, average='macro');
    return score

In [18]:
def createSubmission(filename, y):
    fo = open( filename , 'w' )
    fo.write("ImageId,PredictedClass\n")
    for i in range(y.shape[0]):
        fo.write(str(i)+","+str(y[i])+"\n")
    fo.close()

In [ ]:
models= svmModel(xin,yin)
scoreOVO= cross_val_score(models[0], xin, yin , cv = 2 , scoring='f1_macro')
scoreOVR= cross_val_score(models[1], xin, yin , cv = 2, scoring='f1_macro')
print("F1 score OVO (CV=5): "+str(scoreOVO))
print("F1 score OVR (CV=5): "+str(scoreOVR))

In [6]:
def evaluateModel(kern, C, gamma, xin, yin, xout, yout):
    modelO, modelR= svmModel(kern, xin, yin, C, gamma)
    print(kern + " C = "+str(C)+" and gamma= "+str(gamma))
    scoreO= svmPredict (modelO, xout, yout)
    scoreR= svmPredict (modelR, xout, yout)
    return (scoreO,scoreR)

In [7]:
def tuneParams(kernel, cost, gamma, dim):

    skf = StratifiedKFold(n_splits=4)
    t0= time()
    resultO= 0.0
    resultR= 0.0
    paramsO= ('null', 0.0,0.0, 0)
    paramsR= ('null', 0.0,0.0, 0)
    
    for train, test in skf.split(xin, yin):
        for i in range(C.shape[0]):
            for k in range(len(kernel)):
                for j in range(gamma.shape[0]):
                    for h in range(dim.shape[0]):
                        pca= PCA(n_components = dim[h])
                        pca.fit(xin)
                        xin_new= pca.transform(xin)
                        if(kernel[k]!='linear'):
                            tmp= evaluateModel(kernel[k],C[i],gamma[j], xin_new[train], yin[train], xin_new[test], yin[test])
                            if(resultO<tmp[0]):
                                resultO= tmp[0]
                                paramsO= (kernel[k], C[i], gamma[j], dim[h])
                            if(resultR<tmp[1]):
                                resultR= tmp[1]
                                paramsR= (kernel[k], C[i], gamma[j], dim[h])
                        else:
                            tmp=evaluateModel(kernel[k],C[i],'auto', xin_new[train], yin[train], xin_new[test], yin[test])
                            if(resultO<tmp[0]):
                                resultO= tmp[0]
                                paramsO= (kernel[k], C[i], 0, dim[h])
                            if(resultR<tmp[1]):
                                resultR= tmp[1]
                                paramsR= (kernel[k], C[i], 0, dim[h])
                        
    print("Best obtained for O: "+str(resultO) + " with "+ str(paramsO))
    print("Best obtained for R: " +str(resultR) + " with " +str(paramsR))
    print("Time "+ str(time()-t0))

In [ ]:
C= np.array([1 , 0.1, 0.001, 0.0001, 10])
gamma= np.array([1, 0.1, 0.01, 0.001, 0.0001, 0.00004])
kernel= ['linear', 'rbf']
dim= np.array([966, 400, 350, 200, 100])

tuneParams(kernel, C, gamma, dim)

In [8]:
C= np.array([1 , 0.1, 0.001, 0.0001, 10])
gamma= np.array([1])
kernel= ['linear']
dim= np.array([400, 350, 200, 100])

tuneParams(kernel, C, gamma, dim)

linear C = 1.0 and gamma= auto
linear C = 1.0 and gamma= auto
linear C = 1.0 and gamma= auto
linear C = 1.0 and gamma= auto
linear C = 0.1 and gamma= auto
linear C = 0.1 and gamma= auto
linear C = 0.1 and gamma= auto
linear C = 0.1 and gamma= auto
linear C = 0.001 and gamma= auto
linear C = 0.001 and gamma= auto
linear C = 0.001 and gamma= auto
linear C = 0.001 and gamma= auto
linear C = 0.0001 and gamma= auto
linear C = 0.0001 and gamma= auto
linear C = 0.0001 and gamma= auto
linear C = 0.0001 and gamma= auto
linear C = 10.0 and gamma= auto
linear C = 10.0 and gamma= auto
linear C = 10.0 and gamma= auto
linear C = 10.0 and gamma= auto
linear C = 1.0 and gamma= auto
linear C = 1.0 and gamma= auto
linear C = 1.0 and gamma= auto
linear C = 1.0 and gamma= auto
linear C = 0.1 and gamma= auto
linear C = 0.1 and gamma= auto
linear C = 0.1 and gamma= auto
linear C = 0.1 and gamma= auto
linear C = 0.001 and gamma= auto
linear C = 0.001 and gamma= auto
linear C = 0.001 and gamma= auto
linear C 

In [23]:
pca= PCA(n_components = 200)
pca.fit(xin)
xin_new= pca.transform(xin)
(modelO,modelR)= svmModel ('linear', xin_new,yin ,1 , 'auto')

pca= PCA(n_components = 200)
pca.fit(xout)
xout_new= pca.transform(xout)
ypredict= modelO.predict(xout_new)
createSubmission("submission1NEW.csv", ypredict)

## Statement of Individual Work

Please initial (between the square brackets) one of the following statements.

[FP] I, A0153447A , certify that I have followed the CS 3244 Machine Learning class guidelines for homework assignments.  In particular, I expressly vow that I have followed the Facebook rule in discussing with others in doing the assignment and did not take notes (digital or printed) from the discussions.  

I suggest that I should be graded as follows:

100 marks

### References

I have refered to the following list of people and websites in preparing my homework submission:

http://docs.scipy.org/doc/numpy/reference/,
http://stackoverflow.com/,
http://matplotlib.org/api/pyplot_summary.html,
Textbook: Learning From Data,
Lecture's slides